# TUGAS NLP

## Mohammad Febriyanto
## 18117010

### Berikut ini merupakan algoritma POST TAGGER menggunakan machine learning

In [1]:
def read_dataset(fname):
    sentences = []
    tags = []
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    idx_line = 0
    while idx_line < len(content):
        sent = []
        tag = []
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                sent.append(content_part[0])
                tag.append(content_part[1])
            idx_line = idx_line + 1
        sentences.append(sent)
        tags.append(tag)
        idx_line = idx_line+2        
    return sentences, tags

In [2]:
def features(sentence, index):
    #sentence: [w1, w2, ....], index: the index of the word
    return {
        'word' : sentence[index],
        'is_first': index==0,
        'is_last' : index == len(sentence)-1,
        'is_capitalized' : sentence[index][0].upper == sentence[index][0],
        'is_all_caps' : sentence[index].upper() == sentence[index],
        'is_all_lower' : sentence[index].lower() == sentence[index],
        'prefix-1' : sentence[index][0],
        'prefix-2' : sentence[index][:2],
        'prefix-3' : sentence[index][:3],
        'prefix_4' : sentence[index][:4],
        'suffix-1' : sentence[index][-1],
        'suffix-2' : sentence[index][-2:],
        'suffix-3' : sentence[index][-3:],
        'prev_word' : '' if index == 0 else sentence[index-1],
        'next_word' : '' if index == len(sentence)-1 else sentence[index+1],
        '2-prev-token' : '' if index <= 1 else sentence[index - 2],
        '2-next-token' : '' if index >= len(sentence) - 2 else sentence[index + 2],
        'has_hypen' : '-' in sentence[index],
        'is_numeric' : sentence[index].isdigit(),
        'capitals_inside' : sentence[index][1:].lower() != sentence[index][1:]
    }

In [3]:
sentences,tags = read_dataset('Indonesian_Manually_Tagged_Corpus_ID.tsv')
sentences[0]

['Kera', 'untuk', 'amankan', 'pesta olahraga']

In [4]:
def dataset(tagged_sentences):
    X = []
    
    for tagged in tagged_sentences:
        X.append([features(tagged, index) for index in range(len(tagged))])
        
    return X

In [5]:
sentences = dataset(sentences)

In [6]:
sentences[0]

[{'word': 'Kera',
  'is_first': True,
  'is_last': False,
  'is_capitalized': False,
  'is_all_caps': False,
  'is_all_lower': False,
  'prefix-1': 'K',
  'prefix-2': 'Ke',
  'prefix-3': 'Ker',
  'prefix_4': 'Kera',
  'suffix-1': 'a',
  'suffix-2': 'ra',
  'suffix-3': 'era',
  'prev_word': '',
  'next_word': 'untuk',
  '2-prev-token': '',
  '2-next-token': 'amankan',
  'has_hypen': False,
  'is_numeric': False,
  'capitals_inside': False},
 {'word': 'untuk',
  'is_first': False,
  'is_last': False,
  'is_capitalized': False,
  'is_all_caps': False,
  'is_all_lower': True,
  'prefix-1': 'u',
  'prefix-2': 'un',
  'prefix-3': 'unt',
  'prefix_4': 'untu',
  'suffix-1': 'k',
  'suffix-2': 'uk',
  'suffix-3': 'tuk',
  'prev_word': 'Kera',
  'next_word': 'amankan',
  '2-prev-token': '',
  '2-next-token': 'pesta olahraga',
  'has_hypen': False,
  'is_numeric': False,
  'capitals_inside': False},
 {'word': 'amankan',
  'is_first': False,
  'is_last': False,
  'is_capitalized': False,
  'is_all

In [7]:
#from nltk.tag.util import untag

#split dataset for train and test
cutoff = int(.75 * len(sentences))
X_train = sentences[:cutoff]
y_train = tags[:cutoff]
X_test = sentences[cutoff:]
y_test = tags[cutoff:]

# ML

In [8]:
from sklearn_crfsuite import CRF

model = CRF(algorithm='lbfgs', 
    c1=0.1,
    c2=0.1,
    all_possible_states=True,
    max_iterations=300,
    all_possible_transitions=True)
model.fit(X_train, y_train)

/home/febriyanto/.local/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=True, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=300,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [9]:
from sklearn_crfsuite import metrics

y_pred = model.predict(X_test)
print(metrics.flat_accuracy_score(y_test, y_pred))

0.9662413103513952
